In [0]:
create streaming table sales_stream as 
select *, current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/sales',
  format => 'csv'
);

create streaming table products_stream as 
select *, current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/products',
  format => 'csv'
);

create streaming table customers_stream as 
select *, current_timestamp() as ingestion_date from stream read_files(
  's3://jpmctraining/raw/customers',
  format => 'csv'
)

In [0]:
create streaming table praneeth_silver.sales_silver  
(CONSTRAINT valid_order_id EXPECT (order_id IS NOT NULL) ON VIOLATION DROP ROW) as
select distinct * except(_rescued_data,ingestion_date) from stream praneeth_bronz.sales_stream

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE praneeth_silver.product_silver;

APPLY CHANGES INTO
  praneeth_silver.product_silver
FROM
  stream(praneeth_bronz.products_stream)
KEYS
  (product_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  seqNum
COLUMNS * EXCEPT
  (operation, seqNum,_rescued_data,ingestion_date)
STORED AS
  SCD TYPE 1;

In [0]:
-- Create and populate the target table.
CREATE OR REFRESH STREAMING TABLE praneeth_silver.customer_silver;

APPLY CHANGES INTO
  praneeth_silver.customer_silver
FROM
  stream(praneeth_bronz.customers_stream)
KEYS
  (customer_id)
APPLY AS DELETE WHEN
  operation = "DELETE"
SEQUENCE BY
  sequenceNum
COLUMNS * EXCEPT
  (operation, sequenceNum,_rescued_data,ingestion_date)
STORED AS
  SCD TYPE 2;

In [0]:
create materialized view praneeth_silver.customer_active as 
select * from praneeth_silver.customer_silver where `__END_AT` is null